In [1]:
import plotly.graph_objects as go
import numpy as np
from IPython.display import display


# Define the Variables
N=20 # Number of points in the plot
zeros = np.array([-0.5 + 0.5j, -0.5 - 0.5j]) # Zeros
poles = [0.7*np.exp(1j*np.pi/3), 0.7*np.exp(-1j*np.pi/3)] # Poles

fig = go.Figure() # Create empty figure
def plot_pzmap(poles, zeros, theta_var):
    """plot poles and zeros on the complex plane

    Args:
        poles (list): list of poles
        zeros (list): list of zeros
        theta_var (rad): angle on the unit circle
    """
    # Calculate point on the unit circle
    point = [np.cos(theta_var)+ np.sin(theta_var)*1j]

    
    # Create the theta values of the unit circle
    theta = np.linspace(0, 2*np.pi, 1000)
    # Plot the unit circle
    fig.add_trace(
        go.Scatter(
        x=np.cos(theta),
        y=np.sin(theta),
        name="unit-circle",
        line=dict(
            dash='dash',
            color='black',
        )
        ))
    # Plot the poles and zeros
    fig.add_trace(
        go.Scatter(
        x=np.real(zeros),
        y=np.imag(zeros),
        mode='markers',
        name="zeros",
        visible=True,
        marker=dict(
            color='green',
            size=10,
            symbol='circle-open',
            line=dict(
                width=3
            )
        )
        ))
    fig.add_trace(
        go.Scatter(
        x=np.real(poles),
        y=np.imag(poles),
        mode='markers',
        name="poles",
        marker=dict(
            color='blue',
            size=10,
            symbol='x'
        )
        ))

    # Plot the point on the unit circle
    fig.add_trace(
        go.Scatter(
        x=np.real(point),
        y=np.imag(point),
        mode='markers',
        name="point",
        marker=dict(
            color='red',
            size=10,
        )
        ))


    # Plot the line from the point to the pole
    for pole in poles:
        fig.add_trace(
            go.Scatter(
            x=[np.real(point)[0], np.real(pole)],
            y=[np.imag(point)[0], np.imag(pole)],
            mode='lines',
            name="pole to point",
            line=dict(
                dash='dash',
                color='blue',
                )
            ))
    # Plot the line from the point to the zero
    for zero in zeros:fig.add_trace(
            go.Scatter(
            x=[np.real(point)[0], np.real(zero)],
            y=[np.imag(point)[0], np.imag(zero)],
            name="zero to point",
            mode='lines',
            line=dict(
            dash='dash',
            color='green',
        )
        ))
    # Set the layout
    fig.update_layout(yaxis_range=[-1.5, 1.5], 
                      xaxis_range=[-1.5, 1.5],
                      width=600, 
                      height=600, 
                      template="simple_white"
                      )
    fig.update_xaxes(showgrid=True, zeroline=True)
    fig.update_yaxes(showgrid=True, zeroline=True)


def get_val(poles,zeros,angle_rad):
    """get the value of the function at a given angle

    Args:
        poles (list): list of poles 
        zeros (list): list of zeros
        angle_rad (int): angle on the unit circle

    Returns:
        float: gain of the function
    """
    theta_var = angle_rad
    point = [np.cos(theta_var)+ np.sin(theta_var)*1j]
    distances_pole = [np.abs(point - pole) for pole in poles]
    distances_zero = [np.abs(point - zero) for zero in zeros]
    val=np.prod(distances_zero)/np.prod(distances_pole)
    return val




angle_rad = np.linspace(0, 1*np.pi, N)

for angle in angle_rad:
    value = get_val(poles, zeros, angle)

for angle in angle_rad:
    plot_pzmap(poles, zeros, angle)








# Disaple all traces
for i in range(len(fig.data)):
    fig.data[i].visible = False
    # fig.data[i].title = False
# Enable the first 8 traces (first plot)
for i in range(8):
    fig.data[i].visible = True

# Create and add slider
steps = []
# one plot contains 8 traces
jump=8
for i in range(round(len(fig.data)/jump)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "position: " + str(round(i/round(len(fig.data)/jump-1)*np.pi,2))}],  # layout attribute
        label = str(round(i/round(len(fig.data)/jump-1)*np.pi,2))
    )
    # Enable i'th trace (the i'th plot)
    step["args"][0]["visible"][i*jump+0] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i*jump+1] = True
    step["args"][0]["visible"][i*jump+2] = True
    step["args"][0]["visible"][i*jump+3] = True
    step["args"][0]["visible"][i*jump+4] = True
    step["args"][0]["visible"][i*jump+5] = True
    step["args"][0]["visible"][i*jump+6] = True
    step["args"][0]["visible"][i*jump+7] = True
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Radians: ", "suffix": " rad"},
    pad={"t": 30},
    
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)
# Show the chart
fig.show()
fig.write_html("pole_zero.html")

# export conda enironment: conda env export > environment.yml
